Importing modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Creation of the 3D point coordinate in homogeneous coordinates

In [ ]:
u_w=np.array([[3],[1],[-2],[1]])
#print(u_w)

# First stereo system - Simplified stereo system without noise

# Simulation of the projections on the image buffer

Internal camera parameters common to the two cameras and their intrinsic matrices

In [ ]:
res_x=500
res_y=500
f_x=50
f_y=-50
o_x=res_x/2.
o_y=res_y/2.
# Intrinsic matrix for the left camera
M_int_l=np.array([[f_x,0,o_x],[0,f_y,o_y],[0,0,1]],float)
# Intrinsic matrix for the right camera
M_int_r=M_int_l                                           

Baseline and extrinsic matrices (left camera reference coincides with world camera reference)

In [ ]:
R_l=np.eye(3)                  # Left camera R matrix
t_l=np.zeros((3,1))            # Left camera t vector
# Extrinsic matrix of left camera 
M_ext_l=np.array(np.hstack((R_l,t_l)),dtype=float)   
M_ext_l=M_ext_l

t_x=0.2                        # Baseline
R_r=np.eye(3)                  # Right camera R matrix 
t_r=np.array([[-t_x],[0],[0]]) # Left camera t vector
M_ext_r=np.hstack((R_r,t_r))   # Extrinsic matrix of left camera 

print(M_ext_r)

Camera matrices

In [ ]:
M_l=np.dot(M_int_l,M_ext_l)
M_r=np.dot(M_int_r,M_ext_r)

Projections on the image buffers

In [ ]:
# Homogeneous coordinates of projection on left image buffer
u_im_l=np.dot(M_l,u_w)
# Homogeneous coordinates of projection on right image buffer
u_im_r=np.dot(M_r,u_w)
# Cartesian coordinates of projection on left image buffer
u_im_l_c=(u_im_l/u_im_l[-1])[0:-1]
# Cartesian coordinates of projection on left image buffer
u_im_r_c=(u_im_r/u_im_r[-1])[0:-1] 
print(u_im_r_c)
print(u_im_l_c)

Image buffers

In [ ]:
im_L = np.ones((500,500))  # Left image
im_L[int(u_im_l_c[0])-5:int(u_im_l_c[0])+5,
        int(u_im_l_c[1])-5:int(u_im_l_c[1])+5]=0
im_R = np.ones((500,500))  # Right image
im_R[int(u_im_r_c[0])-5:int(u_im_r_c[0])+5,
        int(u_im_r_c[1])-5:int(u_im_r_c[1])+5]=0

plt.figure(1)
plt.subplot(121)
 # Axis seem transposed
plt.imshow(np.transpose(im_L), cmap='gray')
plt.subplot(122)
# Axis seem transposed
plt.imshow(np.transpose(im_R), cmap='gray') 
plt.show()

# Retrieving 3D coordinates from disparity on the image buffer

In [ ]:
d_im=u_im_l_c[0]-u_im_r_c[0] # Disparity
z=(f_x*t_x)/d_im
x=(z*(u_im_l_c[0]-o_x)/f_x)
y=(z*(u_im_l_c[1]-o_x)/f_y)
print(np.array([x,y,z]))

# Retrieving 3D coordinates using general linear approach

Construction of the A matrix

In [ ]:
A_1=np.hstack((M_l,-u_im_l,np.zeros((3,1))))
A_2=np.hstack((M_r,np.zeros((3,1)),-u_im_r))
A  =np.vstack((A_1,A_2))

SVD

In [ ]:
U,S,V=np.linalg.svd(A)

Solution from the SVD

In [ ]:
u_w_svd_h =V[5,0:4]   # Solution in homogeneous coordinates
u_w_svd_c =(u_w_svd_h/u_w_svd_h[-1])[0:-1]
print(u_w_svd_c.reshape(-1,1))

# Second stereo system - Simplified stereo system with noise

Noisy point positions in the image buffer

In [ ]:
u_im_l_c=u_im_l_c+0.1*np.random.randn(2,1) # Left image point
u_im_r_c=u_im_r_c+0.1*np.random.randn(2,1) # Right image point

3D point using general linear approach

In [ ]:
u_im_l_h = np.vstack((u_im_l_c,1.))
u_im_r_h = np.vstack((u_im_r_c,1.))

A_1=np.hstack((M_l,-u_im_l_h,np.zeros((3,1))))
A_2=np.hstack((M_r,np.zeros((3,1)),-u_im_r_h))
A  =np.vstack((A_1,A_2))
U,S,V=np.linalg.svd(A)
u_w_svd_h  =V[5,0:4]   # Solution in homogeneous coordinates
u_w_svd_c=(u_w_svd_h/u_w_svd_h[-1])[0:-1]
print(u_w_svd_c.reshape(-1,1))

# Third stereo system - Stereo system with rotated right camera and without noise

Baseline and extrinsic matrices (left camera reference coincides with world camera reference)

In [ ]:
R_r=np.array([[np.sqrt(2)/2.,-np.sqrt(2)/2.,0],
              [np.sqrt(2)/2.,np.sqrt(2)/2.,0],
              [0,0,1]])        # Right camera R matrix
M_ext_r=np.hstack((R_r,t_r))   # Extrinsic matrix of left camera 

print(M_ext_r)

Camera matrices

In [ ]:
M_l=np.dot(M_int_l,M_ext_l)
M_r=np.dot(M_int_r,M_ext_r)

Projections on the image buffers

In [ ]:
 # Homogeneous coordinates of projection on left image buffer
u_im_l=np.dot(M_l,u_w)
# Homogeneous coordinates of projection on right image buffer
u_im_r=np.dot(M_r,u_w)
# Cartesian coordinates of projection on left image buffer
u_im_l_c=(u_im_l/u_im_l[-1])[0:-1]
# Cartesian coordinates of projection on left image buffer
u_im_r_c=(u_im_r/u_im_r[-1])[0:-1] 
print(u_im_r_c)

Image buffers

In [ ]:
im_L = np.ones((500,500))  # Left image
im_L[int(u_im_l_c[0])-5:int(u_im_l_c[0])+5,
        int(u_im_l_c[1])-5:int(u_im_l_c[1])+5]=0
im_R = np.ones((500,500))  # Right image
im_R[int(u_im_r_c[0])-5:int(u_im_r_c[0])+5,
        int(u_im_r_c[1])-5:int(u_im_r_c[1])+5]=0

plt.figure(1)
plt.subplot(121)
# Axis seem transposed
plt.imshow(np.transpose(im_L), cmap='gray') 
plt.subplot(122)
# Axis seem transposed
plt.imshow(np.transpose(im_R), cmap='gray') 
plt.show()

3D point using general linear approach

In [ ]:
u_im_l_h = np.vstack((u_im_l_c,1.))
u_im_r_h = np.vstack((u_im_r_c,1.))

A_1=np.hstack((M_l,-u_im_l_h,np.zeros((3,1))))
A_2=np.hstack((M_r,np.zeros((3,1)),-u_im_r_h))
A  =np.vstack((A_1,A_2))
U,S,V=np.linalg.svd(A)
# Solution in homogeneous coordinates
u_w_svd_h  =V[5,0:4]   
u_w_svd_c=(u_w_svd_h/u_w_svd_h[-1])[0:-1]
print(u_w_svd_c.reshape(-1,1))

# Fourth stereo system - Stereo system with rotated right camera and with noise

Noisy point positions in the image buffer

In [ ]:
u_im_l_c=u_im_l_c+0.1*np.random.randn(2,1) # Left image point
u_im_r_c=u_im_r_c+0.1*np.random.randn(2,1) # Right image point

3D point using general linear approach

In [ ]:
u_im_l_h = np.vstack((u_im_l_c,1.))
u_im_r_h = np.vstack((u_im_r_c,1.))

A_1=np.hstack((M_l,-u_im_l_h,np.zeros((3,1))))
A_2=np.hstack((M_r,np.zeros((3,1)),-u_im_r_h))
A  =np.vstack((A_1,A_2))
U,S,V=np.linalg.svd(A)
u_w_svd_h  =V[5,0:4]   # Solution in homogeneous coordinates
u_w_svd_c=(u_w_svd_h/u_w_svd_h[-1])[0:-1]
print(u_w_svd_c.reshape(-1,1))